# Libraries

In [ ]:
# Import necessary libraries

import pandas as pd
import numpy as np
import glob
import re
import cv2 as cv
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import resnet50, inception_v3, densenet
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Model, layers
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

2023-08-09 12:54:46.430417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 12:54:47.844223: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/alvin/anaconda3/envs/asr/lib/python3.9/site-packages/cv2/../../lib64:
2023-08-09 12:54:47.844327: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/alvin/anaconda3/envs/asr/lib/python3.9/site-pack

# Custom Metrics by Nabasa Rodrick

## F1 Score

In [ ]:
from tensorflow.keras.metrics import Precision, Recall

# Instantiate precision and recall metric objects
p = Precision(thresholds=0)
r = Recall(thresholds=0)

def f1_score(y_true, y_pred):
  """
  Function that calculates and returns
  f1_score using Precision and Recall
  """

  def precision(y_true, y_pred):
    """
    Function that calculates and returns precision
    """
    p.update_state(y_true, y_pred)
    return p.result()

  def recall(y_true, y_pred):
    """
    Function that calculates and returns recall
    """
    r.update_state(y_true, y_pred)
    return r.result()

  precision = precision(y_true, y_pred)
  recall = recall(y_true, y_pred)

  result = tf.math.multiply_no_nan(
      tf.constant(2, tf.float32),
      tf.math.divide_no_nan(
          tf.math.multiply_no_nan(
              precision,
              recall
          ),
          tf.math.add(
              precision,
              recall
          )
      )
  )

  return result

# Augmentations

In [ ]:
def random_rotation(img):
  img_shape = img.shape
  img = tf.squeeze(img)

  img = img.numpy().astype(np.uint8)
  img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

  degree = np.random.randint(-15,15,1)[0]

  rows,cols = img.shape
  rotation_matrix = cv.getRotationMatrix2D(((cols-1)/2.0,(rows-1)/2.0), degree, 1)
  img = cv.warpAffine(img, rotation_matrix, (cols,rows))

  img = cv.cvtColor(img, cv.COLOR_GRAY2RGB)
  img = tf.convert_to_tensor(img, dtype=tf.float32)

  img = tf.expand_dims(img, 0)
  img.set_shape(img_shape)

  return img

In [ ]:
def hist_eq(img):
  img_shape = img.shape
  img = tf.squeeze(img)

  img = img.numpy().astype(np.uint8)
  img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

  contrast = np.random.uniform(1.0,2.0)
  clahe = cv.createCLAHE(clipLimit=contrast, tileGridSize=(16,16))
  img = clahe.apply(img)

  img = cv.cvtColor(img, cv.COLOR_GRAY2RGB)
  img = tf.convert_to_tensor(img, dtype=tf.float32)

  img = tf.expand_dims(img, 0)
  img.set_shape(img_shape)

  return img

In [ ]:
# Smoothing to remove noise
def smoothing(img):
  img_shape = img.shape
  img = tf.squeeze(img)

  img = img.numpy().astype(np.uint8)

  img = cv.GaussianBlur(img,(3,3),0)

  img = tf.convert_to_tensor(img, dtype=tf.float32)

  img = tf.expand_dims(img, 0)
  img.set_shape(img_shape)

  return img

In [ ]:
@tf.function
def augmentation(img):
  img = tf.py_function(hist_eq, [img], tf.float32)
  img = tf.py_function(random_rotation, [img], tf.float32)

  img = tf.reshape(img, (1,96,103,channels))

  return img

# Global Variables

In [ ]:
#Define global variables
channels = 4
image_size = (96,103)
image_shape = image_size + (channels,)
batch_size = 64

rng = tf.random.Generator.from_seed(42, alg='philox')